In [1]:
import fastf1
import matplotlib.pyplot as plt
import seaborn as sns
import fastf1.plotting
import matplotlib as mpl
import numpy as np
from matplotlib.collections import LineCollection
from matplotlib import colormaps

In [4]:
class FastF1:

    def __init__(self, x):
        self.x = x
        self.schedule = fastf1.get_event_schedule(self.x)
        self.session = None  # Will store loaded race session

    def get_event_details(self):
        return self.schedule

    def get_event_round(self, round_number):
        return self.schedule[self.schedule['RoundNumber'] == round_number][['EventDate', 'EventName', 'Session1', 'Session2', 'Session3', 'Session4', 'Session5'
        ]]
        
    def get_race_details(self, event_name, day):
        session = fastf1.get_session(self.x, event_name, day)
        session.load()
        self.session = session

        return [
            'Choose the DataSet to Analyse the Race',
            "1. Laps",
            "2. Results",
            "3. Weather",
            "4. Track Status",
            "5. Messages"
        ]

    def get_dataset(self, option):
        if (option==1):
            time_columns = [
            'Time', 'LapTime', 'PitOutTime', 'PitInTime',
            'Sector1Time', 'Sector2Time', 'Sector3Time',
            'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
            'LapStartTime']
            
        for col in time_columns:
            self.session.laps[col]=self.session.laps[col].dt.total_seconds()

        dataset_map = {
            1: self.session.laps,
            2: self.session.results,
            3: self.session.weather_data,
            4: self.session.track_status,
            5: self.session.race_control_messages,
        }

        return dataset_map.get(option, "🚫 Invalid selection. Choose a number from 1 to 5.")

    def get_circuit_image(self, event_name, day, colormap=mpl.cm.plasma):
        session = fastf1.get_session(self.x, event_name, day)
        session.load()
        self.session = session

        drivers = session.laps['Driver'].unique()
        driver_map = {i + 1: driver for i, driver in enumerate(drivers)}

        print("\n🏎️ Available Drivers:")
        for number, driver in driver_map.items():
            print(f"{number}: {driver}")

        try:
            choice = int(input("\n👉 Enter the number of the driver to visualize: "))
            selected_driver = driver_map.get(choice)
            if not selected_driver:
                print("🚫 Invalid driver selection.")
                return
        except ValueError:
            print("⚠️ Please enter a valid numeric option.")
            return

        lap = session.laps.pick_drivers(selected_driver).pick_fastest()
        x = lap.telemetry['X']
        y = lap.telemetry['Y']
        color = lap.telemetry['Speed']

        points = np.array([x, y]).T.reshape(-1, 1, 2)
        segments = np.concatenate([points[:-1], points[1:]], axis=1)

        fig, ax = plt.subplots(sharex=True, sharey=True, figsize=(12, 6.75))
        fig.suptitle(
            f"{session.event['EventName']} {self.x} - {selected_driver} - Speed",
            size=14, y=0.87
        )
        plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.12)
        ax.axis('off')

        ax.plot(x, y, color='black', linestyle='-', linewidth=16, zorder=0)

        norm = plt.Normalize(color.min(), color.max())
        lc = LineCollection(segments, cmap=colormap, norm=norm, linestyle='-', linewidth=5)
        lc.set_array(color)
        ax.add_collection(lc)

        cbaxes = fig.add_axes([0.25, 0.05, 0.5, 0.05])
        normlegend = mpl.colors.Normalize(vmin=color.min(), vmax=color.max())
        mpl.colorbar.ColorbarBase(cbaxes, norm=normlegend, cmap=colormap, orientation="horizontal")

        plt.show()

    def gear_shifts_on_track(self, event_name, day):
        session = fastf1.get_session(self.x, event_name, day)
        session.load()
        self.session = session

        drivers = session.laps['Driver'].unique()
        driver_map = {i + 1: driver for i, driver in enumerate(drivers)}

        print("\n Available Drivers for Gear Visualization:")
        for num, name in driver_map.items():
            print(f"{num}: {name}")

        try:
            choice = int(input("\n👉 Enter the number of the driver to visualize: "))
            selected_driver = driver_map.get(choice)
            if not selected_driver:
                print("🚫 Invalid driver selection.")
                return
        except ValueError:
            print("⚠️ Please enter a valid numeric option.")
            return

        lap = session.laps.pick_drivers(selected_driver).pick_fastest()
        telemetry = lap.get_telemetry()

        x = telemetry['X'].values
        y = telemetry['Y'].values
        gear = telemetry['nGear'].to_numpy().astype(float)

        points = np.array([x, y]).T.reshape(-1, 1, 2)
        segments = np.concatenate([points[:-1], points[1:]], axis=1)

        cmap = colormaps['Paired']
        lc = LineCollection(segments, cmap=cmap, norm=plt.Normalize(1, cmap.N + 1))
        lc.set_array(gear)
        lc.set_linewidth(4)

        fig, ax = plt.subplots(figsize=(10, 5.75))
        fig.suptitle(
            f"Fastest Lap Gear Shift Visualization\n"
            f"{selected_driver} - {session.event['EventName']} {self.x}",
            size=24, y=0.97
        )
        ax.add_collection(lc)
        ax.axis('equal')
        ax.axis('off')

        cb = fig.colorbar(lc, ax=ax, orientation='horizontal', label='Gear',
                          boundaries=np.arange(1, 10))
        cb.set_ticks(np.arange(1.5, 9.5))
        cb.set_ticklabels(np.arange(1, 9))

        plt.show()
    
    def drivers_fastest_lap(self, event_name, day):
        session= fastf1.get_session(self.x,event_name,day)
        session.load()
        
        fastest_lap= session.laps.loc[session.laps['LapTime'].idxmin()] 
        return fastest_lap[['Driver','LapNumber','LapTime']]

    def top5_drivers_laptime(self, event_name,day):
        session=fastf1.get_session(self.x,event_name,day)
        session.load()
        Avg_laptime= (
           session.laps.groupby('Driver')['LapTime'].mean()
           .reset_index(name='Avg_LapTime')
           .sort_values(by='Avg_LapTime',ascending=True).head(5)
       ) 
        return Avg_laptime



In [6]:
t3= FastF1(2021)

t3.get_event_details()

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,...,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport
0,0,Bahrain,Bahrain,FORMULA 1 ARAMCO PRE-SEASON TESTING 2021,2021-03-14,Pre-Season Test,testing,Practice 1,2021-03-12 10:00:00+03:00,2021-03-12 07:00:00,...,Practice 3,2021-03-14 10:00:00+03:00,2021-03-14 07:00:00,None,NaT,NaT,None,NaT,NaT,False
1,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2021,2021-03-28,Bahrain Grand Prix,conventional,Practice 1,2021-03-26 14:30:00+03:00,2021-03-26 11:30:00,...,Practice 3,2021-03-27 15:00:00+03:00,2021-03-27 12:00:00,Qualifying,2021-03-27 18:00:00+03:00,2021-03-27 15:00:00,Race,2021-03-28 18:00:00+03:00,2021-03-28 15:00:00,True
2,2,Italy,Imola,FORMULA 1 PIRELLI GRAN PREMIO DEL MADE IN ITAL...,2021-04-18,Emilia Romagna Grand Prix,conventional,Practice 1,2021-04-16 11:00:00+02:00,2021-04-16 09:00:00,...,Practice 3,2021-04-17 11:00:00+02:00,2021-04-17 09:00:00,Qualifying,2021-04-17 14:00:00+02:00,2021-04-17 12:00:00,Race,2021-04-18 15:00:00+02:00,2021-04-18 13:00:00,True
3,3,Portugal,Portimão,FORMULA 1 HEINEKEN GRANDE PRÉMIO DE PORTUGAL 2021,2021-05-02,Portuguese Grand Prix,conventional,Practice 1,2021-04-30 11:30:00+01:00,2021-04-30 10:30:00,...,Practice 3,2021-05-01 12:00:00+01:00,2021-05-01 11:00:00,Qualifying,2021-05-01 15:00:00+01:00,2021-05-01 14:00:00,Race,2021-05-02 15:00:00+01:00,2021-05-02 14:00:00,True
4,4,Spain,Barcelona,FORMULA 1 ARAMCO GRAN PREMIO DE ESPAÑA 2021,2021-05-09,Spanish Grand Prix,conventional,Practice 1,2021-05-07 11:30:00+02:00,2021-05-07 09:30:00,...,Practice 3,2021-05-08 12:00:00+02:00,2021-05-08 10:00:00,Qualifying,2021-05-08 15:00:00+02:00,2021-05-08 13:00:00,Race,2021-05-09 15:00:00+02:00,2021-05-09 13:00:00,True
5,5,Monaco,Monte Carlo,FORMULA 1 GRAND PRIX DE MONACO 2021,2021-05-23,Monaco Grand Prix,conventional,Practice 1,2021-05-20 11:30:00+02:00,2021-05-20 09:30:00,...,Practice 3,2021-05-22 12:00:00+02:00,2021-05-22 10:00:00,Qualifying,2021-05-22 15:00:00+02:00,2021-05-22 13:00:00,Race,2021-05-23 15:00:00+02:00,2021-05-23 13:00:00,True
6,6,Azerbaijan,Baku,FORMULA 1 AZERBAIJAN GRAND PRIX 2021,2021-06-06,Azerbaijan Grand Prix,conventional,Practice 1,2021-06-04 12:30:00+04:00,2021-06-04 08:30:00,...,Practice 3,2021-06-05 13:00:00+04:00,2021-06-05 09:00:00,Qualifying,2021-06-05 16:00:00+04:00,2021-06-05 12:00:00,Race,2021-06-06 16:00:00+04:00,2021-06-06 12:00:00,True
7,7,France,Le Castellet,FORMULA 1 EMIRATES GRAND PRIX DE FRANCE 2021,2021-06-20,French Grand Prix,conventional,Practice 1,2021-06-18 11:30:00+02:00,2021-06-18 09:30:00,...,Practice 3,2021-06-19 12:00:00+02:00,2021-06-19 10:00:00,Qualifying,2021-06-19 15:00:00+02:00,2021-06-19 13:00:00,Race,2021-06-20 15:00:00+02:00,2021-06-20 13:00:00,True
8,8,Austria,Spielberg,FORMULA 1 BWT GROSSER PREIS DER STEIERMARK 2021,2021-06-27,Styrian Grand Prix,conventional,Practice 1,2021-06-25 11:30:00+02:00,2021-06-25 09:30:00,...,Practice 3,2021-06-26 12:00:00+02:00,2021-06-26 10:00:00,Qualifying,2021-06-26 15:00:00+02:00,2021-06-26 13:00:00,Race,2021-06-27 15:00:00+02:00,2021-06-27 13:00:00,True
9,9,Austria,Spielberg,FORMULA 1 BWT GROSSER PREIS VON ÖSTERREICH 2021,2021-07-04,Austrian Grand Prix,conventional,Practice 1,2021-07-02 11:30:00+02:00,2021-07-02 09:30:00,...,Practice 3,2021-07-03 12:00:00+02:00,2021-07-03 10:00:00,Qualifying,2021-07-03 15:00:00+02:00,2021-07-03 13:00:00,Race,2021-07-04 15:00:00+02:00,2021-07-04 13:00:00,True
